In [2]:
import numpy as np
import pandas as pd
import math

In [13]:
class SOM:
    def __init__(self, m, n, c=2, l=0.5, lamb=1, sigma=1) -> None:
        self.m = m
        self.n = n
        self.l = l
        self.c = c
        self.lamb = lamb
        self.sigma = sigma
        self.weights = np.random.rand(c, n)
    
    def get_winner(self, sample):
        min_ind = -1
        min_dist = float('inf')
        for i in range(self.c):
            d = 0
            for j in range(self.n):
                d += math.pow((sample[j] - self.weights[i][j]), 2)
            if d < min_dist:
                min_ind = i
                min_dist = d
        return min_ind
    
    def update(self, sample, j, t):
        rate = self.__activation(sample, j, t)
        self.weights[j] = self.weights[j] + rate * (sample - self.weights[j])
    
    def train(self, data, epochs=3):
        for t in range(epochs):
            for j in range(self.m):
                sample = data[j]
                win = self.get_winner(sample)
                self.update(sample, win, t)

    def __activation(self, sample, j, t):
        dist = 0
        for i in range(self.n):
            dist += math.pow((sample[i] - self.weights[j][i]), 2)
        exp_factor = math.exp(-1 * t / self.lamb)
        sigma = self.sigma * exp_factor

        theta = math.exp(-1 * dist / (2 * sigma * sigma))
        l_factor = self.l * exp_factor

        return theta * l_factor

In [16]:
data = np.array([
    np.array([1, 1, 0, 0]),
    np.array([0, 0, 0, 1]),
    np.array([1, 0, 0, 1]),
    np.array([0, 0, 1, 1])
])
som = SOM(len(data), len(data[0]))
som.train(data)
som.weights

array([[0.81902177, 0.22797657, 0.00749438, 0.69887551],
       [0.42630734, 0.72720811, 0.94925461, 0.39581912]])

In [17]:
s = np.array([0,0,0,1])
som.get_winner(s)

0

In [19]:
data = pd.read_csv("../data/diabetes.csv")
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [30]:
X = data.drop("Outcome", axis=1).to_numpy()
X

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [32]:
som = SOM(len(X), len(X[0]))
som.train(X)
som.weights

array([[0.94218344, 0.73149066, 0.75670744, 0.63022054, 0.7236691 ,
        0.54941598, 0.09448798, 0.66184653],
       [0.78191158, 0.861638  , 0.12123146, 0.75672769, 0.66099373,
        0.42125267, 0.99774117, 0.0140891 ]])